In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
import numpy as np
import os
from pathlib import Path
import math
from datetime import date
from getpass import getuser
import re
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid", context="talk")
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

In [2]:
def obtener_rutas_usuario(base_local: Path = None, base_remoto: str = None, usar_remoto: bool = False):
    """
    Devuelve las rutas de datos para el usuario actual, usando pathlib.
    Ajusta la ruta si estás ejecutando desde 'notebooks/', considerando la ruta relativa.

    Parámetros:
    - base_local: ruta base local donde se encuentran los datos, considerando la estructura cookiecutter (por defecto: '../data/')
    - base_remoto: ruta base remota tipo GCP, cuando estas en la VM
    - usar_remoto: True para rutas de cloud (base_remoto), False para rutas locales (base_local)

    Retorna:
    - dict con rutas: rawData, externalData, interimData, processedData
    """
    if base_local is None:
        try:
            base_local = Path(__file__).resolve().parent.parent / 'data'
        except NameError:
            base_local = Path.cwd()
            if 'notebooks' in base_local.parts:
                base_local = base_local.parent
            base_local = base_local / 'data'

    if base_remoto is None:
        base_remoto = 'gs://default-bucket/ScoreCI'

    if usar_remoto:
        rutas = {
            'rawData': f'{base_remoto}/raw/',
            'externalData': f'{base_remoto}/external/',
            'interimData': f'{base_remoto}/interim/',
            'processedData': f'{base_remoto}/processed/'
        }
    else:
        rutas = {
            'rawData': base_local / 'raw',
            'externalData': base_local / 'external',
            'interimData': base_local / 'interim',
            'processedData': base_local / 'processed'
        }

    return rutas

In [3]:
rutas = obtener_rutas_usuario()

In [4]:
df = pd.read_parquet(os.path.join(rutas.get('rawData'), 'ent-prd-sandbox-fdo-bucket_ScoreCI_raw_BaseAnaliticaCliente_ScoreCI_05052025_dic24v6.parquet'))

In [5]:
dfAsesor = pd.read_parquet(os.path.join(rutas.get('rawData'), 'ent-prd-sandbox-fdo-bucket_ScoreCI_raw_BaseAnaliticaAsesor_ScoreCI_30042025_dic24v6.parquet'))

In [6]:
pd.pivot_table(data=df, index='ADVISOR_ID', columns='FECHA_CORTE', 
               values = ['FLAG_TR','CONTRACT_ID'], 
               aggfunc={'FLAG_TR':'sum', 'CONTRACT_ID':'count'},
              fill_value=0).reset_index()

ADVISOR_ID CONTRACT_ID                          FLAG_TR  \
FECHA_CORTE             2024-10-31 2024-11-30 2024-12-31 2024-10-31   
0              1949827         119        113        120          0   
1              1949838         242        245        248          3   
2              1949853         144        142        141          0   
3              1950001         130        139        145          0   
4              1950569         164        168        172          0   
...                ...         ...        ...        ...        ...   
2032          94223258           0          0        135          0   
2033          94223260           0          0        119          0   
2034          94223263           0          0        130          0   
2035          94261339           0          0          2          0   
2036          94339256           0          0         56          0   

                                   
FECHA_CORTE 2024-11-30 2024-12-31  
0                    0          0  
1                    0          0  
2                    0          0  
3                    0          0  
4                    0          0  
...                ...        ...  
2032                 0        123  
2033                 0        117  
2034                 0        124  
2035                 0          0  
2036                 0         47  

[2037 rows x 7 columns]

In [7]:
ls_cortes = [-math.inf,1, 8,61 ,math.inf]
ls_etiquetas = ['a.0', 'b.1-7', 'c.8-60', 'd.60+']
df['rangoAtraso'] = pd.cut(df['DIAS_ATRASO_ACTUAL'], bins = ls_cortes, labels = ls_etiquetas, right = False)

In [9]:
pd.pivot_table(data=df, index='FECHA_CORTE', columns='FLAG_TR', values = 'CONTRACT_ID', aggfunc='count')

FLAG_TR,0,1
FECHA_CORTE,,
2024-10-31,240158,15921
2024-11-30,250055,17253
2024-12-31,260085,12942


In [10]:
pd.pivot_table(data=df, index='FECHA_CORTE', columns='FLAG_TR', values = 'MONTO_CARTERA', aggfunc='sum')

FLAG_TR,0,1
FECHA_CORTE,,
2024-10-31,1.153965e+10,6.481521e+08
2024-11-30,1.227485e+10,7.244325e+08
2024-12-31,1.295481e+10,5.732957e+08


In [12]:
df.groupby('FLAG_TR')['ADVISOR_ID'].count()

FLAG_TR
0    750298
1     46116
Name: ADVISOR_ID, dtype: int64

In [13]:
dfTo = df[df.FECHA_CORTE == date(2024, 12, 31)]

In [16]:
colsTr = ['ASESOR_ID','Tot_ctes','Tot_ctes_tr','Tot_cartera','Tot_cartera_tr']

df_tr = pd.pivot_table(data=dfTo,
               index='ADVISOR_ID', columns='FLAG_TR', 
               values = ['CONTRACT_ID','MONTO_CARTERA'], 
               aggfunc={'MONTO_CARTERA':'sum', 'CONTRACT_ID':'count'},
               fill_value=0
              ).reset_index()

df_tr.columns = colsTr
df_tr.head()

,ASESOR_ID,Tot_ctes,Tot_ctes_tr,Tot_cartera,Tot_cartera_tr
0,1949827,120,0,5253864.58,0.0
1,1949838,248,0,10712964.79,0.0
2,1949853,141,0,6512461.18,0.0
3,1950001,145,0,7415418.69,0.0
4,1950569,172,0,9671375.57,0.0


In [18]:
df_tr['porc_ctes_tr'] = df_tr['Tot_ctes_tr']/df_tr['Tot_ctes']

In [19]:
df_tr['porc_ctes_tr'].describe(percentiles=[.9,.95,.97,.99])

count    2037.000000
mean             inf
std              NaN
min         0.000000
50%         0.000000
90%         0.022326
95%         0.442455
97%         5.215873
99%        53.880000
max              inf
Name: porc_ctes_tr, dtype: float64

In [21]:
ls_cortes_tr = [-math.inf, 0.022326,0.442455,5.215873,53.880000,math.inf]
ls_etiquetas = ['b.90', 'c.95', 'd.97','e.99','f.99+']
df_tr['rango_porc_ctes_tr'] = pd.cut(df_tr['porc_ctes_tr'], bins = ls_cortes_tr, labels = ls_etiquetas, right = False)

In [22]:
df_tr.groupby('rango_porc_ctes_tr').agg({'ASESOR_ID':'count', 'porc_ctes_tr':['min','max']}).reset_index()#.to_clipboard()

rango_porc_ctes_tr ASESOR_ID porc_ctes_tr            
                         count          min         max
0               b.90      1833     0.000000    0.022059
1               c.95       102     0.022727    0.426357
2               d.97        40     0.506849    5.142857
3               e.99        41     5.222222   45.666667
4              f.99+         3    58.500000  102.000000

In [23]:
df_tr[df_tr['rango_porc_ctes_tr'].isnull()]

,ASESOR_ID,Tot_ctes,Tot_ctes_tr,Tot_cartera,Tot_cartera_tr,porc_ctes_tr,rango_porc_ctes_tr
13,1953602,0,1,0.0,38757.18,inf,NaN
36,2950827,0,95,0.0,4345218.91,inf,NaN
47,4492730,0,133,0.0,4209650.38,inf,NaN
51,5171460,0,15,0.0,315365.55,inf,NaN
60,5760645,0,6,0.0,106681.85,inf,NaN
83,6976798,0,29,0.0,953246.20,inf,NaN
109,8416873,0,18,0.0,606520.23,inf,NaN
136,9330371,0,1,0.0,34449.76,inf,NaN
155,9975239,0,1,0.0,178812.83,inf,NaN
167,11853141,0,1,0.0,37859.67,inf,NaN


In [ ]:
colscr = ['ASESOR_ID','CR0','CR1a7','CR8a60','CR60+']

df_cr = pd.pivot_table(data=dfTo,
               index='ADVISOR_ID', columns='rangoAtraso', 
               values = ['MONTO_CARTERA'], 
               aggfunc={'MONTO_CARTERA':'sum'},
               fill_value=0
              ).reset_index()

df_cr.columns = colscr
df_cr.head()

In [ ]:
df_cr['CARTERA_ASESOR'] = df_cr.iloc[:,1:].sum(axis = 1)

In [ ]:
df_cr['porc_cr'] = df_cr['CR8a60']/df_cr['CARTERA_ASESOR']

In [ ]:
df_cr.head()

In [ ]:
ls_seg = [-math.inf,0.016, 0.0250,math.inf]
ls_etiquetas = ['a.Baja', 'b.Media', 'c.Alta']
df_cr['rango_porc_cr'] = pd.cut(df_cr['porc_cr'], bins = ls_seg, labels = ls_etiquetas, right = False)

In [ ]:
df_cr.groupby('rango_porc_cr')['ASESOR_ID'].count()

In [ ]:
df_tr_cr = df_tr.merge(df_cr, on='ASESOR_ID', how='inner')

In [ ]:
dfAsesor[dfAsesor.EMPLOYEE_ID==18798275]

In [ ]:
dfAsesor1 = dfAsesor[dfAsesor['POSITION_DESC']=='ASESOR CI']

In [ ]:
df_tr_cr = df_tr_cr.merge(dfAsesor1, left_on='ASESOR_ID', right_on='EMPLOYEE_ID', how='inner')

In [ ]:
df_tr_cr[(df_tr_cr['rango_porc_cr']=='a.Baja') & (df_tr_cr['rango_porc_ctes_tr']=='e.90')]

In [ ]:
pd.pivot_table(
    data=df_tr_cr,
    index='rango_porc_cr',
    columns='rango_porc_ctes_tr',
    values='ASESOR_ID',
    aggfunc='count'
).reset_index()#.to_clipboard()

In [ ]:
dfTo.head()

In [ ]:
pd.pivot_table(
    data=dfTo[dfTo['FLAG_FPD']!= -1],
    index='ADVISOR_ID',
    columns='FLAG_TR',
    values=['CONTRACT_ID','FLAG_FPD'],
    aggfunc={'CONTRACT_ID':'count','FLAG_FPD':'sum'},
    fill_value=0
).reset_index()

In [ ]:
dfTo.FLAG_FPD.value_counts()

In [ ]:
pd.pivot_table(
    data=dfTo,
    index='FLAG_TR',
    columns='rangoAtraso',
    values='ADVISOR_ID',
    aggfunc='count',
    margins=True
).reset_index()

# Asesor

In [ ]:
dfAsesor.head()

In [ ]:
dfAsesor[dfAsesor.FLAG_NUEVO==1]